In [1]:
import pandas as pd
import chromadb
from typing import List, Dict

def prepare_nutrition_documents(csv_path: str) -> Dict:
    """
    Convert nutrition CSV into ChromaDB-ready documents.
    Each food item becomes a searchable document.
    """
    df = pd.read_csv(csv_path)
    
    documents = []
    metadatas = []
    ids = []
    
    for index, row in df.iterrows():
        # Create rich document text for semantic search
        document_text = f"""
        Food: {row['FoodItem']}
        Category: {row['FoodCategory']}
        Nutritional Information:
        - Calories: {row['Cals_per100grams']} per 100g
        - Energy: {row['KJ_per100grams']} kJ per 100g
        - Serving size reference: {row['per100grams']}
        
        This is a {row['FoodCategory'].lower()} food item that provides {row['Cals_per100grams']} calories per 100 grams.
        """.strip()
        
        # Rich metadata for filtering and exact lookups
        metadata = {
            "food_item": row['FoodItem'].lower(),
            "food_category": row['FoodCategory'].lower(),
            "calories_per_100g": float(row['Cals_per100grams']) if pd.notna(row['Cals_per100grams']) else 0,
            "kj_per_100g": float(row['KJ_per100grams']) if pd.notna(row['KJ_per100grams']) else 0,
            "serving_info": row['per100grams'],
            # Add searchable keywords
            "keywords": f"{row['FoodItem'].lower()} {row['FoodCategory'].lower()}".replace(' ', '_')
        }
        
        documents.append(document_text)
        metadatas.append(metadata)
        ids.append(f"food_{index}")
    
    return {
        "documents": documents,
        "metadatas": metadatas, 
        "ids": ids
    }

def setup_nutrition_chromadb(csv_path: str, collection_name: str = "nutrition_db"):
    """
    Create and populate ChromaDB collection with nutrition data.
    """
    # Initialize ChromaDB
    client = chromadb.Client()
    
    # Create collection (delete if exists)
    try:
        client.delete_collection(collection_name)
    except:
        pass
    
    collection = client.create_collection(
        name=collection_name,
        metadata={"description": "Nutrition database with calorie and food information"}
    )
    
    # Prepare documents
    data = prepare_nutrition_documents(csv_path)
    
    # Add to ChromaDB
    collection.add(
        documents=data["documents"],
        metadatas=data["metadatas"],
        ids=data["ids"]
    )
    
    print(f"Added {len(data['documents'])} food items to ChromaDB collection '{collection_name}'")
    return collection


def nutrition_lookup_tool(query: str, collection, max_results: int = 3) -> str:
    """
    Tool function that your nutrition agent can call.
    """
    results = collection.query(
        query_texts=[query],
        n_results=max_results
    )
    
    if not results['documents'][0]:
        return f"No nutrition information found for: {query}"
    
    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results['documents'][0]):
        metadata = results['metadatas'][0][i]
        food_item = metadata['food_item'].title()
        calories = metadata['calories_per_100g']
        category = metadata['food_category'].title()
        
        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )
    
    return "Nutrition Information:\n" + "\n".join(formatted_results)



RuntimeError: [91mYour system has an unsupported version of sqlite3. Chroma                     requires sqlite3 >= 3.35.0.[0m
[94mPlease visit                     https://docs.trychroma.com/troubleshooting#sqlite to learn how                     to upgrade.[0m

In [1]:
    collection = setup_nutrition_chromadb("data/calories.csv")
    
    # Run example queries
    example_nutrition_queries(collection)
    
    # Test the tool function
    print("\n=== TOOL FUNCTION EXAMPLES ===\n")
    print("Query: 'apple calories'")
    print(nutrition_lookup_tool("apple calories", collection))
    
    print("\nQuery: 'high protein foods'") 
    print(nutrition_lookup_tool("high protein foods", collection))

NameError: name 'setup_nutrition_chromadb' is not defined

In [5]:
import chromadb

chroma_client = chromadb.PersistentClient()
collection = chroma_client.create_collection(name="my_collection")

collection.add(
    ids=["id1", "id2"],
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ]
)

In [6]:
results = collection.query(
    query_texts=["This is a query document about pineapple"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[0.1824142336845398, 1.004446268081665]]}
